<a href="https://colab.research.google.com/github/vishal-burman/PyTorch-Architectures/blob/master/misc/SentenceSimilarity(DistilRoBERTa_DistilRoBERTa).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install transformers
! pip install datasets

     |████████████████████████████████| 2.6 MB 3.9 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 636 kB 59.7 MB/s 
     |████████████████████████████████| 3.3 MB 42.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 264 kB 5.1 MB/s 
     |████████████████████████████████| 76 kB 4.9 MB/s 
     |████████████████████████████████| 243 kB 46.6 MB/s 
     |████████████████████████████████| 118 kB 55.5 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [3]:
! git clone https://github.com/vishal-burman/PyTorch-Architectures.git
%cd PyTorch-Architectures/

Cloning into 'PyTorch-Architectures'...
remote: Enumerating objects: 3954, done.
remote: Counting objects: 100% (1321/1321), done.
remote: Compressing objects: 100% (581/581), done.
remote: Total 3954 (delta 801), reused 1181 (delta 690), pack-reused 2633
Receiving objects: 100% (3954/3954), 10.37 MiB | 21.44 MiB/s, done.
Resolving deltas: 100% (2239/2239), done.
/content/PyTorch-Architectures


In [4]:
import random
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel

In [5]:
dataset = load_dataset("quora")

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/559 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

Dataset quora downloaded and prepared to /root/.cache/huggingface/datasets/quora/default/0.0.0/36ba4cd42107f051a158016f1bea6ae3f4685c5df843529108a54e42d86c1e04. Subsequent calls will reuse this data.


In [6]:
train_p = []
train_n = []
test_list = []
count_p, count_n = 0, 0
for idx, sample in enumerate(dataset["train"]):
  text_1, text_2 = sample["questions"]["text"][0], sample["questions"]["text"][1]
  if len(train_p) < 10000 and sample["is_duplicate"]:
    train_p.append((text_1, text_2, 1))
  elif len(train_n) < 10000 and not sample["is_duplicate"]:
    train_n.append((text_1, text_2, 0))
  elif len(test_list) < 10000:
    is_duplicate = 1 if sample["is_duplicate"] else 0
    test_list.append((text_1, text_2, is_duplicate))
train_list = []
train_list.extend(train_p)
train_list.extend(train_n)
random.shuffle(train_list)
print(f"No. of Train Samples: {len(train_list)} || No. of Test Samples: {len(test_list)}")

No. of Train Samples: 20000 || No. of Test Samples: 10000


In [7]:
path_str = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(path_str)
model = AutoModel.from_pretrained(path_str)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
class CustomDataset(Dataset):
  def __init__(self, tokenizer, list_samples: list, max_input_length: int = 16):
    self.tokenizer = tokenizer
    self.list_samples = list_samples
    self.max_input_length = max_input_length
  
  def __len__(self,):
    return len(self.list_samples)
  
  def __getitem__(self, idx):
    sample = self.list_samples[idx]
    return {
        'text_1': sample[0],
        'text_2': sample[1],
        'is_duplicate': sample[2],
    }
  
  def collate_fn(self, batch):
    text_1 = []
    text_2 = []
    labels = []
    for sample in batch:
      text_1.append(sample["text_1"])
      text_2.append(sample["text_2"])
      labels.append(sample["is_duplicate"])
    tokens_1 = self.tokenizer(text_1,
                              max_length=self.max_input_length,
                              padding=True,
                              truncation=True,
                              return_tensors="pt",
                              )
    tokens_2 = self.tokenizer(text_2,
                              max_length=self.max_input_length,
                              padding=True,
                              truncation=True,
                              return_tensors="pt",
                              )
    ids_1, att_1 = tokens_1["input_ids"], tokens_1["attention_mask"]
    ids_2, att_2 = tokens_2["input_ids"], tokens_2["attention_mask"]
    labels = torch.tensor(labels, dtype=torch.long)
    return {
        "input_ids_1": ids_1,
        "attention_mask_1": att_1,
        "input_ids_2": ids_2,
        "attention_mask_2": att_2,
        "labels": labels,
    }

In [9]:
class Attention(nn.Module):
  def __init__(self, in_size: int = 768, hidden_size: int = 512):
    super().__init__()
    self.W = nn.Linear(in_size, hidden_size)
    self.V = nn.Linear(hidden_size, 1)
    self.dropout = nn.Dropout(0.3)
  
  def forward(self, x):
    x = torch.tanh(self.W(x))
    score = self.V(x)
    attention_weights = score.softmax(dim=1)
    context_vector = x * attention_weights
    context_vector = torch.sum(context_vector, dim=1)
    output = self.dropout(context_vector)
    return output

In [20]:
class SentenceSimilarity(nn.Module):
  def __init__(self, encoder, in_size: int = 768, hidden_size: int = 768):
    super().__init__()
    self.encoder = encoder
    self.attention = Attention(in_size, hidden_size)
    self.ff = nn.Linear(hidden_size * 2, 2)
  
  def forward(self,
              input_ids_1,
              attention_mask_1,
              input_ids_2,
              attention_mask_2,
              labels=None,
              ):
    
    outputs_1 = self.encoder(input_ids=input_ids_1,
                               attention_mask=attention_mask_1)
    outputs_2 = self.encoder(input_ids=input_ids_2,
                               attention_mask=attention_mask_2)
    
    enc_weights_1 = self.attention(outputs_1.last_hidden_state)
    enc_weights_2 = self.attention(outputs_2.last_hidden_state)

    output = torch.cat([enc_weights_1, enc_weights_2], dim=1)

    logits = self.ff(output)
    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(logits.size(0), -1), labels.view(-1))
    return (loss, logits)